In [1]:
!pip install keras-tcn
!pip install -U finance-datareader

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [2]:
import numpy as np
import pandas as pd
import FinanceDataReader as fdr
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from tcn import TCN

/home/work/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.0' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
2023-01-27 01:57:32.805226: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-27 01:57:32.969465: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-27 01:57:35.311714: W tensorflow/compiler/xla/stream_executor/platform/default/

In [3]:
kospi_train = fdr.DataReader('KS11', '1990-01-01','2017-12-31')
kospi_train['Change1'] = kospi_train['Close']/kospi_train['Close'].shift(1) - 1
kospi_train['Change7'] = kospi_train['Close']/kospi_train['Close'].shift(7) - 1
kospi_train['Change15'] = kospi_train['Close']/kospi_train['Close'].shift(15) - 1
kospi_train['Change30'] = kospi_train['Close']/kospi_train['Close'].shift(30) - 1
print(kospi_train.head())
print(kospi_train.columns)

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
1996-12-11  705.989990  709.479980  704.429993  704.679993  704.679993   
1996-12-12  705.109985  706.010010  688.739990  689.380005  689.380005   
1996-12-13  690.440002  695.719971  677.640015  689.070007  689.070007   
1996-12-16  686.969971  686.969971  667.710022  673.919983  673.919983   
1996-12-17  675.349976  680.090027  660.390015  663.349976  663.349976   

             Volume   Change1  Change7  Change15  Change30  
Date                                                        
1996-12-11  28000.0       NaN      NaN       NaN       NaN  
1996-12-12  25900.0 -0.021712      NaN       NaN       NaN  
1996-12-13  26500.0 -0.000450      NaN       NaN       NaN  
1996-12-16  22800.0 -0.021986      NaN       NaN       NaN  
1996-12-17  31600.0 -0.015684      NaN       NaN       NaN  
Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'V

In [4]:
kospi_train = kospi_train.dropna()
print(kospi_train.head())
print(kospi_train.columns)

                  Open        High         Low       Close   Adj Close  \
Date                                                                     
1997-01-24  669.030029  679.559998  658.179993  679.559998  679.559998   
1997-01-27  684.309998  685.460022  664.679993  664.700012  664.700012   
1997-01-28  661.659973  677.630005  653.419983  662.849976  662.849976   
1997-01-29  663.330017  672.900024  654.950012  663.559998  663.559998   
1997-01-30  663.650024  679.039978  662.270020  676.520020  676.520020   

             Volume   Change1   Change7  Change15  Change30  
Date                                                         
1997-01-24  35500.0  0.003915 -0.028867  0.039416 -0.013801  
1997-01-27  28300.0 -0.021867 -0.066249  0.060347 -0.013681  
1997-01-28  30400.0 -0.002783 -0.054665  0.084772 -0.000754  
1997-01-29  30200.0  0.001071 -0.076413  0.067830  0.003281  
1997-01-30  28800.0  0.019531 -0.020303  0.057277 -0.020913  
Index(['Open', 'High', 'Low', 'Close', 'Adj Clo

In [5]:
kospi_train_close = kospi_train['Close']
kospi_train_c1 = kospi_train['Change1']
kospi_train_c7 = kospi_train['Change7']
kospi_train_c15 = kospi_train['Change15']
kospi_train_c30 = kospi_train['Change30']
print(kospi_train_close, kospi_train_c1)

Date
1997-01-24     679.559998
1997-01-27     664.700012
1997-01-28     662.849976
1997-01-29     663.559998
1997-01-30     676.520020
                 ...     
2017-12-21    2429.830078
2017-12-22    2440.540039
2017-12-26    2427.340088
2017-12-27    2436.669922
2017-12-28    2467.489990
Name: Close, Length: 4668, dtype: float64 Date
1997-01-24    0.003915
1997-01-27   -0.021867
1997-01-28   -0.002783
1997-01-29    0.001071
1997-01-30    0.019531
                ...   
2017-12-21   -0.019649
2017-12-22    0.004408
2017-12-26   -0.005409
2017-12-27    0.003844
2017-12-28    0.012648
Name: Change1, Length: 4668, dtype: float64


In [6]:
kospi_test = fdr.DataReader('KS11', '2018-01-01')
kospi_test['Change1'] = kospi_test['Close']/kospi_test['Close'].shift(1) - 1
kospi_test['Change7'] = kospi_test['Close']/kospi_test['Close'].shift(7) - 1
kospi_test['Change15'] = kospi_test['Close']/kospi_test['Close'].shift(15) - 1
kospi_test['Change30'] = kospi_test['Close']/kospi_test['Close'].shift(30) - 1
print(kospi_test.head())
print(kospi_test.columns)

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2018-01-02  2474.860107  2481.020020  2465.939941  2479.649902  2479.649902   
2018-01-03  2484.629883  2493.399902  2481.909912  2486.350098  2486.350098   
2018-01-04  2502.500000  2502.500000  2466.449951  2466.459961  2466.459961   
2018-01-05  2476.850098  2497.520020  2475.510010  2497.520020  2497.520020   
2018-01-08  2510.699951  2515.370117  2494.179932  2513.280029  2513.280029   

            Volume   Change1  Change7  Change15  Change30  
Date                                                       
2018-01-02  262200       NaN      NaN       NaN       NaN  
2018-01-03  331100  0.002702      NaN       NaN       NaN  
2018-01-04  333800 -0.008000      NaN       NaN       NaN  
2018-01-05  308800  0.012593      NaN       NaN       NaN  
2018-01-08  311400  0.006310      NaN       NaN       NaN  
Index(['Open', 'High', 'Lo

In [7]:
kospi_test = kospi_test.dropna()
print(kospi_test.head())
print(kospi_test.columns)

                   Open         High          Low        Close    Adj Close  \
Date                                                                          
2018-02-13  2402.889893  2421.800049  2388.540039  2395.189941  2395.189941   
2018-02-14  2412.469971  2426.629883  2408.020020  2421.830078  2421.830078   
2018-02-19  2452.520020  2455.120117  2428.149902  2442.820068  2442.820068   
2018-02-20  2433.350098  2438.479980  2411.229980  2415.120117  2415.120117   
2018-02-21  2417.729980  2432.449951  2404.020020  2429.649902  2429.649902   

            Volume   Change1   Change7  Change15  Change30  
Date                                                        
2018-02-13  423400  0.004113 -0.051556 -0.055748 -0.034061  
2018-02-14  305700  0.011122 -0.028061 -0.045772 -0.025950  
2018-02-19  322100  0.008667 -0.004276 -0.046604 -0.009585  
2018-02-20  421500 -0.011339  0.007744 -0.062002 -0.032993  
2018-02-21  432300  0.006016  0.009150 -0.064868 -0.033275  
Index(['Open', 'Hig

In [8]:
kospi_test_close = kospi_test['Close']
kospi_test_c1 = kospi_test['Change1']
kospi_test_c7 = kospi_test['Change7']
kospi_test_c15 = kospi_test['Change15']
kospi_test_c30 = kospi_test['Change30']
print(kospi_test_close, kospi_test_c1)

Date
2018-02-13    2395.189941
2018-02-14    2421.830078
2018-02-19    2442.820068
2018-02-20    2415.120117
2018-02-21    2429.649902
                 ...     
2023-01-19    2380.340088
2023-01-20    2395.260010
2023-01-25    2428.570068
2023-01-26    2468.649902
2023-01-27    2483.199951
Name: Close, Length: 1218, dtype: float64 Date
2018-02-13    0.004113
2018-02-14    0.011122
2018-02-19    0.008667
2018-02-20   -0.011339
2018-02-21    0.006016
                ...   
2023-01-19    0.005075
2023-01-20    0.006268
2023-01-25    0.013907
2023-01-26    0.016503
2023-01-27    0.005894
Name: Change1, Length: 1218, dtype: float64


In [9]:
kospi_train_close = kospi_train_close.values
kospi_train_c1 = 100 * kospi_train_c1.values
kospi_train_c7 = 100 * kospi_train_c7.values
kospi_train_c15 = 100 * kospi_train_c15.values
kospi_train_c30 = 100 * kospi_train_c30.values

kospi_test_close = kospi_test_close.values
kospi_test_c1 = 100 * kospi_test_c1.values
kospi_test_c7 = 100 * kospi_test_c7.values
kospi_test_c15 = 100 * kospi_test_c15.values
kospi_test_c30 = 100 * kospi_test_c30.values
print(kospi_test_c1)

[0.41125768 1.11223484 0.86669953 ... 1.39066564 1.65034703 0.58939297]


In [10]:
lookback_window = 60
x_train_1, y_train_1 = [], []
x_train_7, y_train_7 = [], []
x_train_15, y_train_15 = [], []
x_train_30, y_train_30 = [], []

In [11]:
for i in range(lookback_window, len(kospi_train_close)-1):
    x_train_1.append(kospi_train_close[i - lookback_window:i])
    y_train_1.append(kospi_train_c1[i])

for i in range(lookback_window, len(kospi_train_close)-6):
    x_train_7.append(kospi_train_close[i - lookback_window:i])
    y_train_7.append(kospi_train_c7[i+6])

for i in range(lookback_window, len(kospi_train_close)-14):
    x_train_15.append(kospi_train_close[i - lookback_window:i])
    y_train_15.append(kospi_train_c15[i+14])

for i in range(lookback_window, len(kospi_train_close)-29):
    x_train_30.append(kospi_train_close[i - lookback_window:i])
    y_train_30.append(kospi_train_c30[i+29])

In [12]:
for index in range(len(y_train_1)):
    if y_train_1[index]>= -0.01 and y_train_1[index] < 0.01:
        y_train_1[index] = 'neutral'
    elif y_train_1[index] >= 0.01:
        y_train_1[index] = 'increase'
    else:
        y_train_1[index] = 'decrease'
        
for index in range(len(y_train_7)):
    if y_train_7[index]>= -0.01 and y_train_7[index] < 0.01:
        y_train_7[index] = 'neutral'
    elif y_train_7[index] >= 0.01:
        y_train_7[index] = 'increase'
    else:
        y_train_7[index] = 'decrease'

for index in range(len(y_train_15)):
    if y_train_15[index]>= -0.01 and y_train_15[index] < 0.01:
        y_train_15[index] = 'neutral'
    elif y_train_15[index] >= 0.01:
        y_train_15[index] = 'increase'
    else:
        y_train_15[index] = 'decrease'

for index in range(len(y_train_30)):
    if y_train_30[index]>= -0.01 and y_train_30[index] < 0.01:
        y_train_30[index] = 'neutral'
    elif y_train_30[index] >= 0.01:
        y_train_30[index] = 'increase'
    else:
        y_train_30[index] = 'decrease'

In [13]:
x_train_1 = np.array(x_train_1)
y_train_1 = np.array(y_train_1)
x_train_1 = x_train_1.reshape((4607, -1, 1))
#y_train_1 = y_train_1.reshape((-1, 1))

x_train_7 = np.array(x_train_7)
y_train_7 = np.array(y_train_7)
x_train_7 = x_train_7.reshape((4602, -1, 1))
#y_train_7 = y_train_7.reshape((-1, 1))

x_train_15 = np.array(x_train_15)
y_train_15 = np.array(y_train_15)
x_train_15 = x_train_15.reshape((4594, -1, 1))
#y_train_15 = y_train_15.reshape((-1, 1))

x_train_30 = np.array(x_train_30)
y_train_30 = np.array(y_train_30)
x_train_30 = x_train_30.reshape((4579, -1, 1))
#y_train_30 = y_train_30.reshape((-1, 1))

print(x_train_1.shape, x_train_7.shape, x_train_15.shape, x_train_30.shape)
print(y_train_1.shape, y_train_7.shape, y_train_15.shape, y_train_30.shape)

(4607, 60, 1) (4602, 60, 1) (4594, 60, 1) (4579, 60, 1)
(4607,) (4602,) (4594,) (4579,)


In [26]:
x_test_1, y_test_1 = [], []
x_test_7, y_test_7 = [], []
x_test_15, y_test_15 = [], []
x_test_30, y_test_30 = [], []

In [27]:
for i in range(lookback_window, len(kospi_test_close)-1):
    x_test_1.append(kospi_test_close[i - lookback_window:i])
    y_test_1.append(kospi_test_c1[i])

for i in range(lookback_window, len(kospi_test_close)-6):
    x_test_7.append(kospi_test_close[i - lookback_window:i])
    y_test_7.append(kospi_test_c7[i+6])

for i in range(lookback_window, len(kospi_test_close)-14):
    x_test_15.append(kospi_test_close[i - lookback_window:i])
    y_test_15.append(kospi_test_c15[i+14])

for i in range(lookback_window, len(kospi_test_close)-29):
    x_test_30.append(kospi_test_close[i - lookback_window:i])
    y_test_30.append(kospi_test_c30[i+29])

In [28]:
for index in range(len(y_test_1)):
    if y_test_1[index]>= -0.01 and y_test_1[index] < 0.01:
        y_test_1[index] = 'neutral'
    elif y_test_1[index] >= 0.01:
        y_test_1[index] = 'increase'
    else:
        y_test_1[index] = 'decrease'
        
for index in range(len(y_test_7)):
    if y_test_7[index]>= -0.01 and y_test_7[index] < 0.01:
        y_test_7[index] = 'neutral'
    elif y_test_7[index] >= 0.01:
        y_test_7[index] = 'increase'
    else:
        y_test_7[index] = 'decrease'

for index in range(len(y_test_15)):
    if y_test_15[index]>= -0.01 and y_test_15[index] < 0.01:
        y_test_15[index] = 'neutral'
    elif y_test_15[index] >= 0.01:
        y_test_15[index] = 'increase'
    else:
        y_test_15[index] = 'decrease'

for index in range(len(y_test_30)):
    if y_test_30[index]>= -0.01 and y_test_30[index] < 0.01:
        y_test_30[index] = 'neutral'
    elif y_test_30[index] >= 0.01:
        y_test_30[index] = 'increase'
    else:
        y_test_30[index] = 'decrease'

In [31]:
x_test_1 = np.array(x_test_1)
y_test_1 = np.array(y_test_1)
x_test_1 = x_test_1.reshape((1157, -1, 1))
#y_test_1 = y_test_1.reshape((-1, 1))

x_test_7 = np.array(x_test_7)
y_test_7 = np.array(y_test_7)
x_test_7 = x_test_7.reshape((1152, -1, 1))
#y_test_7 = y_test_7.reshape((-1, 1))

x_test_15 = np.array(x_test_15)
y_test_15 = np.array(y_test_15)
x_test_15 = x_test_15.reshape((1144, -1, 1))
#y_test_15 = y_test_15.reshape((-1, 1))

x_test_30 = np.array(x_test_30)
y_test_30 = np.array(y_test_30)
x_test_30 = x_test_30.reshape((1129, -1, 1))
#y_test_30 = y_test_30.reshape((-1, 1))

In [32]:
print(x_test_1.shape, x_test_7.shape, x_test_15.shape, x_test_30.shape)
print(y_test_1.shape, y_test_7.shape, y_test_15.shape, y_test_30.shape)

(1157, 60, 1) (1152, 60, 1) (1144, 60, 1) (1129, 60, 1)
(1157,) (1152,) (1144,) (1129,)


In [33]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
encoder = LabelEncoder()
encoder.fit(y_train_1)

y_train_1_e = to_categorical(encoder.transform(y_train_1))
y_train_7_e = to_categorical(encoder.transform(y_train_7))
y_train_15_e = to_categorical(encoder.transform(y_train_15))
y_train_30_e = to_categorical(encoder.transform(y_train_30))

y_test_1_e = to_categorical(encoder.transform(y_test_1))
y_test_7_e = to_categorical(encoder.transform(y_test_7))
y_test_15_e = to_categorical(encoder.transform(y_test_15))
y_test_30_e = to_categorical(encoder.transform(y_test_30))

In [34]:
model_1 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=2,
        use_skip_connections=False,
        use_batch_norm=False,
        use_weight_norm=False,
        use_layer_norm=False
        ),
    Dense(3, activation='softmax')
])

model_1.summary()
model_1.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
####
model_7 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=2,
        use_skip_connections=False,
        use_batch_norm=False,
        use_weight_norm=False,
        use_layer_norm=False
        ),
    Dense(3, activation='softmax')
])

model_7.summary()
model_7.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
####
model_15 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=2,
        use_skip_connections=False,
        use_batch_norm=False,
        use_weight_norm=False,
        use_layer_norm=False
        ),
    Dense(3, activation='softmax')
])

model_15.summary()
model_15.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
####
model_30 = Sequential([
    TCN(input_shape=(lookback_window, 1),
        kernel_size=2,
        use_skip_connections=False,
        use_batch_norm=False,
        use_weight_norm=False,
        use_layer_norm=False
        ),
    Dense(3, activation='softmax')
])

model_30.summary()
model_30.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

2023-01-27 02:02:07.062244: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/compat/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib:/usr/local/cuda/lib64:/usr/local/nvidia/lib64:/usr/include/x86_64-linux-gnu
2023-01-27 02:02:07.062290: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-01-27 02:02:07.062318: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:163] no NVIDIA GPU device is present: /dev/nvidia0 does not exist
2023-01-27 02:02:07.062761: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in othe

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn (TCN)                   (None, 64)                91136     
                                                                 
 dense (Dense)               (None, 3)                 195       
                                                                 
Total params: 91,331
Trainable params: 91,331
Non-trainable params: 0
_________________________________________________________________
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 tcn_1 (TCN)                 (None, 64)                91136     
                                                                 
 dense_1 (Dense)             (None, 3)                 195       
                                                                 
Total params: 91,331
Trainable

In [35]:
print('Train model_1...')
model_1.fit(x_train_1, y_train_1_e, epochs=100, verbose=1)
p_test_1 = model_1.predict(x_test_1)

print('Train model_7...')
model_7.fit(x_train_7, y_train_7_e, epochs=100, verbose=1)
p_test_7 = model_7.predict(x_test_7)

print('Train model_15...')
model_15.fit(x_train_15, y_train_15_e, epochs=100, verbose=1)
p_test_15 = model_15.predict(x_test_15)

print('Train model_30...')
model_30.fit(x_train_30, y_train_30_e, epochs=100, verbose=1)
p_test_30 = model_30.predict(x_test_30)

Train model_1...
Epoch 1/100
144/144 [==============================] - 5s 18ms/step - loss: 1658.4349 - accuracy: 0.4925
Epoch 2/100
144/144 [==============================] - 3s 18ms/step - loss: 332.4568 - accuracy: 0.4845
Epoch 3/100
144/144 [==============================] - 2s 17ms/step - loss: 249.7529 - accuracy: 0.4801
Epoch 4/100
144/144 [==============================] - 2s 17ms/step - loss: 134.4137 - accuracy: 0.4903
Epoch 5/100
144/144 [==============================] - 2s 17ms/step - loss: 124.4942 - accuracy: 0.4966
Epoch 6/100
144/144 [==============================] - 2s 17ms/step - loss: 170.4035 - accuracy: 0.4949
Epoch 7/100
144/144 [==============================] - 2s 17ms/step - loss: 171.9065 - accuracy: 0.4884
Epoch 8/100
144/144 [==============================] - 2s 17ms/step - loss: 79.7803 - accuracy: 0.4867
Epoch 9/100
144/144 [==============================] - 2s 17ms/step - loss: 70.0810 - accuracy: 0.4973
Epoch 10/100
144/144 [==========================

144/144 [==============================] - 2s 17ms/step - loss: 19.2677 - accuracy: 0.4979
Epoch 80/100
144/144 [==============================] - 2s 17ms/step - loss: 16.9228 - accuracy: 0.4873
Epoch 81/100
144/144 [==============================] - 2s 17ms/step - loss: 14.0968 - accuracy: 0.4951
Epoch 82/100
144/144 [==============================] - 2s 17ms/step - loss: 17.8363 - accuracy: 0.5014
Epoch 83/100
144/144 [==============================] - 2s 17ms/step - loss: 12.1821 - accuracy: 0.4990
Epoch 84/100
144/144 [==============================] - 2s 17ms/step - loss: 16.0372 - accuracy: 0.4719
Epoch 85/100
144/144 [==============================] - 2s 17ms/step - loss: 18.9951 - accuracy: 0.4888
Epoch 86/100
144/144 [==============================] - 2s 17ms/step - loss: 13.2859 - accuracy: 0.4882
Epoch 87/100
144/144 [==============================] - 2s 17ms/step - loss: 13.5395 - accuracy: 0.4888
Epoch 88/100
144/144 [==============================] - 2s 17ms/step - loss: 

144/144 [==============================] - 2s 17ms/step - loss: 21.2884 - accuracy: 0.5117
Epoch 58/100
144/144 [==============================] - 2s 17ms/step - loss: 16.9476 - accuracy: 0.5102
Epoch 59/100
144/144 [==============================] - 2s 17ms/step - loss: 11.1681 - accuracy: 0.5096
Epoch 60/100
144/144 [==============================] - 2s 17ms/step - loss: 13.3524 - accuracy: 0.5128
Epoch 61/100
144/144 [==============================] - 2s 17ms/step - loss: 20.8893 - accuracy: 0.4944
Epoch 62/100
144/144 [==============================] - 2s 17ms/step - loss: 21.8020 - accuracy: 0.5187
Epoch 63/100
144/144 [==============================] - 2s 17ms/step - loss: 13.8816 - accuracy: 0.4948
Epoch 64/100
144/144 [==============================] - 2s 17ms/step - loss: 13.7641 - accuracy: 0.5087
Epoch 65/100
144/144 [==============================] - 2s 17ms/step - loss: 15.0375 - accuracy: 0.5122
Epoch 66/100
144/144 [==============================] - 2s 17ms/step - loss: 

144/144 [==============================] - 2s 17ms/step - loss: 26.5548 - accuracy: 0.5050
Epoch 36/100
144/144 [==============================] - 2s 17ms/step - loss: 27.2659 - accuracy: 0.5194
Epoch 37/100
144/144 [==============================] - 2s 17ms/step - loss: 28.7759 - accuracy: 0.5028
Epoch 38/100
144/144 [==============================] - 2s 17ms/step - loss: 21.1595 - accuracy: 0.5057
Epoch 39/100
144/144 [==============================] - 2s 17ms/step - loss: 22.8817 - accuracy: 0.5137
Epoch 40/100
144/144 [==============================] - 2s 17ms/step - loss: 15.3306 - accuracy: 0.5102
Epoch 41/100
144/144 [==============================] - 2s 17ms/step - loss: 12.7609 - accuracy: 0.5239
Epoch 42/100
144/144 [==============================] - 2s 17ms/step - loss: 15.8354 - accuracy: 0.5059
Epoch 43/100
144/144 [==============================] - 2s 17ms/step - loss: 19.8316 - accuracy: 0.5172
Epoch 44/100
144/144 [==============================] - 2s 17ms/step - loss: 

144/144 [==============================] - 2s 17ms/step - loss: 67.4146 - accuracy: 0.5016
Epoch 14/100
144/144 [==============================] - 2s 17ms/step - loss: 62.6942 - accuracy: 0.5027
Epoch 15/100
144/144 [==============================] - 2s 17ms/step - loss: 35.6179 - accuracy: 0.5185
Epoch 16/100
144/144 [==============================] - 2s 17ms/step - loss: 64.8364 - accuracy: 0.5152
Epoch 17/100
144/144 [==============================] - 2s 17ms/step - loss: 52.4862 - accuracy: 0.5106
Epoch 18/100
144/144 [==============================] - 2s 17ms/step - loss: 46.1806 - accuracy: 0.5230
Epoch 19/100
144/144 [==============================] - 2s 17ms/step - loss: 43.8266 - accuracy: 0.5180
Epoch 20/100
144/144 [==============================] - 2s 17ms/step - loss: 44.9085 - accuracy: 0.5246
Epoch 21/100
144/144 [==============================] - 2s 17ms/step - loss: 55.9164 - accuracy: 0.5285
Epoch 22/100
144/144 [==============================] - 2s 17ms/step - loss: 

144/144 [==============================] - 2s 16ms/step - loss: 12.9015 - accuracy: 0.5130
Epoch 93/100
144/144 [==============================] - 2s 16ms/step - loss: 11.3753 - accuracy: 0.5051
Epoch 94/100
144/144 [==============================] - 2s 16ms/step - loss: 9.4979 - accuracy: 0.5178
Epoch 95/100
144/144 [==============================] - 2s 16ms/step - loss: 10.2781 - accuracy: 0.5080
Epoch 96/100
144/144 [==============================] - 2s 16ms/step - loss: 12.3818 - accuracy: 0.5219
Epoch 97/100
144/144 [==============================] - 2s 16ms/step - loss: 10.5338 - accuracy: 0.5126
Epoch 98/100
144/144 [==============================] - 2s 16ms/step - loss: 8.2527 - accuracy: 0.5178
Epoch 99/100
144/144 [==============================] - 2s 16ms/step - loss: 7.8729 - accuracy: 0.5115
Epoch 100/100
36/36 [==============================] - 1s 20ms/step


In [36]:
model_1.save('/home/work/nlp/TCN_models/classification_model/v5/kospi_1day_classification_v5.h5')
model_7.save('/home/work/nlp/TCN_models/classification_model/v5/kospi_7days_classification_v5.h5')
model_15.save('/home/work/nlp/TCN_models/classification_model/v5/kospi_15days_classification_v5.h5')
model_30.save('/home/work/nlp/TCN_models/classification_model/v5/kospi_30days_classification_v5.h5')

In [38]:
model_1.evaluate(x_test_1, y_test_1_e)[1]
model_7.evaluate(x_test_7, y_test_7_e)[1]
model_15.evaluate(x_test_15, y_test_15_e)[1]
model_30.evaluate(x_test_30, y_test_30_e)[1]

36/36 [==============================] - 1s 20ms/step - loss: 9.9685 - accuracy: 0.4925


0.49247121810913086